# Using Spark for Feature Extraction:
In this notebook, we use PySpark and Hadoop to speed up feature extraction, namely the Google Books API calls to retrieve each book's description, page count, and categories. Unfortunately, we kept running into API rate limit issues that parallelization only exacerbated, so we had to abandon using Spark for the feature extraction step. This notebook is here as reference.

In [ ]:
import os
import time
import requests

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import ArrayType, IntegerType, StringType, StructType, StructField

In [3]:
# Set environment variables
# python_exec = path\to\python
os.environ["PYSPARK_PYTHON"] = python_exec
os.environ["PYSPARK_DRIVER_PYTHON"] = python_exec

# hadoop_home = path\to\hadoop
os.environ["HADOOP_HOME"] = hadoop_home
os.environ["hadoop.home.dir"] = hadoop_home

In [4]:
# Initialize Spark session
spark = SparkSession.builder \
        .appName("BookDatasetFeatureExtraction") \
        .config("spark.executorEnv.PYSPARK_PYTHON", python_exec) \
        .config("spark.executorEnv.PYSPARK_DRIVER_PYTHON", python_exec) \
        .getOrCreate()

In [5]:
# Fetch book information from Google Books API
def get_book_info(isbn):
    time.sleep(5)
    headers = {
        'content-type': "application/json"
    }
    url = f"https://www.googleapis.com/books/v1/volumes?q=isbn:{isbn}"
    response = requests.get(url, headers=headers)
    data = response.json()

    if "items" in data:
        volume_info = data["items"][0]["volumeInfo"]
        description = volume_info.get("description", "")
        page_count = volume_info.get("pageCount", 0)
        categories = volume_info.get("categories", [])

    else:
        description = ""
        page_count = 0
        categories = []

    return (description, page_count, categories)

In [6]:
# Register the UDF
schema = StructType([
    StructField("Description", StringType(), True),
    StructField("Page-Count", IntegerType(), True),
    StructField("Categories", ArrayType(StringType()), True)
])
get_book_info_udf = udf(lambda isbn: get_book_info(isbn), schema)
spark.udf.register("get_book_info_udf", get_book_info_udf)

In [7]:
# Load dataset
books = spark.read.csv("data\Books.csv", inferSchema=True, header=True)

In [8]:
books.drop("Image-URL-S", "Image-URL-M").withColumnRenamed("Image-URL-L", "Image-URL")

DataFrame[ISBN: string, Book-Title: string, Book-Author: string, Year-Of-Publication: string, Publisher: string, Image-URL: string]

In [9]:
# Add new columns with book info
books_with_info = books.withColumn("Book-Info", get_book_info_udf(books["ISBN"]))

books_with_info = books_with_info.withColumn("Description", books_with_info["Book-Info.Description"]) \
                                 .withColumn("Page-Count", books_with_info["Book-Info.Page-Count"]) \
                                 .withColumn("Categories", books_with_info["Book-Info.Categories"]) \

books_with_info = books_with_info.drop("Book-Info")

In [10]:
books_with_info.show(truncate=False, vertical=True)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [10]:
books_with_info.write.json("data/books_with_info", mode="overwrite")

Py4JJavaError: An error occurred while calling o83.json.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 3.0 failed 1 times, most recent failure: Lost task 4.0 in stage 3.0 (TID 18) (DESKTOP-78FT3K2.lan executor driver): org.apache.spark.SparkException: [TASK_WRITE_FAILED] Task failed while writing rows to file:/C:/Users/TVan/Desktop/workspace/Book-Recommendation/feature_extraction/data/books_with_info.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.taskFailedWhileWritingRowsError(QueryExecutionErrors.scala:774)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.executeTask(FileFormatWriter.scala:420)
	at org.apache.spark.sql.execution.datasources.WriteFilesExec.$anonfun$doExecuteWrite$1(WriteFiles.scala:100)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2(RDD.scala:893)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2$adapted(RDD.scala:893)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPa

In [13]:
spark.stop()